### Imports

In [1]:
!pip install pydantic[email] fastapi

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.6/313.6 kB 11.5 MB/s eta 0:00:00


In [2]:
from datetime import datetime, timedelta
from typing import Literal, Optional
from uuid import uuid4, UUID
from pydantic import BaseModel, Field, field_validator, model_validator

### Classe AIAgent

In [31]:
class AIAgent(BaseModel):
    id: UUID = Field(default_factory=uuid4)
    name: str = Field(..., min_length=2)
    model_type: Literal['LLM', 'SLM', 'Recommender', 'Retriever']
    context_window: int = Field(..., gt=0, description="Max tokens the agent can process")
    max_tasks: int = Field(default=3, ge=1, le=10)
    active: bool = True
    current_tasks: int = Field(default=0, le=10)

    @field_validator("name")
    def validate_name(v: str) -> str:
        if not v.replace(" ", "").isalpha():
            raise ValueError("Name must only contain letters and spaces")
        return v

Classe AITask

In [32]:
class AITask(BaseModel):
    id: UUID = Field(default_factory=uuid4)
    task_type: Literal['chat', 'summarization', 'retrieval', 'recommendation']
    assigned_to: UUID
    required_context: int = Field(..., gt=0)
    deadline: datetime = Field(...)

    @model_validator(mode="before")
    @classmethod
    def validate_deadline(cls, values):
        if "deadline" in values and values["deadline"] < datetime.now():
            raise ValueError("Deadline must be in the future")
        return values

In [33]:
agents: list[AIAgent] = []
tasks: list[AITask] = []

### Funções

In [34]:
def create_agent(name: str, model_type: str, context_window: int, max_tasks: int = 3) -> AIAgent:
    agent = AIAgent(name=name, model_type=model_type, context_window=context_window, max_tasks=max_tasks)
    agents.append(agent)
    return agent

def assign_task(agent_id: UUID, task_type: str, required_context: int, deadline: datetime) -> AITask:
    agent = next((a for a in agents if a.id == agent_id), None)
    if not agent:
        raise ValueError("Agent not found")
    if not agent.active:
        raise ValueError("Cannot assign tasks to inactive agent")
    if agent.current_tasks >= agent.max_tasks:
        raise ValueError("Agent is overloaded")
    if required_context > agent.context_window:
        raise ValueError("Required context exceeds agent capacity")

    task = AITask(task_type=task_type, assigned_to=agent.id, required_context=required_context, deadline=deadline)
    tasks.append(task)
    agent.current_tasks += 1
    return task

### Tests

In [35]:
# Criando agentes IA
agent1 = create_agent("Amelia", "LLM", context_window=4096)
agent2 = create_agent("Evah", "Retriever", context_window=1024, max_tasks=2)

print("AI Agents created:\n", agents)

# Atribuindo tarefas válidas
task1 = assign_task(agent1.id, "chat", required_context=1000, deadline=datetime.now() + timedelta(days=1))
task2 = assign_task(agent2.id, "retrieval", required_context=512, deadline=datetime.now() + timedelta(hours=8))

print("\nTasks assigned:\n", tasks)

# Tentando erro: tarefa acima da janela de contexto
try:
    assign_task(agent2.id, "retrieval", required_context=2048, deadline=datetime.now() + timedelta(days=1))
except ValueError as e:
    print("\n[Erro esperado - contexto excedido]:", e)

# Tentando erro: deadline no passado
try:
    assign_task(agent1.id, "chat", required_context=1000, deadline=datetime.now() - timedelta(days=1))
except ValueError as e:
    print("\n[Erro esperado - deadline passada]:", e)

# Tentando erro: nome inválido
try:
    create_agent("Agent#99!", "SLM", context_window=2048)
except ValueError as e:
    print("\n[Erro esperado - nome inválido]:", e)

AI Agents created:
 [AIAgent(id=UUID('48960414-ca73-4fa9-820b-cd674b0333e5'), name='Amelia', model_type='LLM', context_window=4096, max_tasks=3, active=True, current_tasks=0), AIAgent(id=UUID('cfce29f8-6064-4f38-8fc2-8bc0ea173449'), name='Evah', model_type='Retriever', context_window=1024, max_tasks=2, active=True, current_tasks=0)]

Tasks assigned:
 [AITask(id=UUID('698ab89f-4753-4465-98b0-49ca268cbdc3'), task_type='chat', assigned_to=UUID('48960414-ca73-4fa9-820b-cd674b0333e5'), required_context=1000, deadline=datetime.datetime(2025, 4, 4, 18, 8, 26, 522534)), AITask(id=UUID('008fce01-c38b-444e-80a8-53320166079f'), task_type='retrieval', assigned_to=UUID('cfce29f8-6064-4f38-8fc2-8bc0ea173449'), required_context=512, deadline=datetime.datetime(2025, 4, 4, 2, 8, 26, 522765))]

[Erro esperado - contexto excedido]: Required context exceeds agent capacity

[Erro esperado - deadline passada]: 1 validation error for AITask
  Value error, Deadline must be in the future [type=value_error, inp

#### Serialization

In [36]:
import json
print("\nAgentes (JSON):\n", json.dumps([a.model_dump(mode='json') for a in agents], indent=2))


Agentes (JSON):
 [
  {
    "id": "48960414-ca73-4fa9-820b-cd674b0333e5",
    "name": "Amelia",
    "model_type": "LLM",
    "context_window": 4096,
    "max_tasks": 3,
    "active": true,
    "current_tasks": 1
  },
  {
    "id": "cfce29f8-6064-4f38-8fc2-8bc0ea173449",
    "name": "Evah",
    "model_type": "Retriever",
    "context_window": 1024,
    "max_tasks": 2,
    "active": true,
    "current_tasks": 1
  }
]
